### **Data Collection**

**Purpose:** This program collects hand gesture data for training a gesture recognition model.
<br>
**Functionality:**
- Captures hand landmarks using Mediapipe.
- Saves the landmarks as .npy files in a structured directory for later use in model training.
- Provides visual feedback by displaying the captured video feed with overlaid landmarks.
<br><br>
If you want to run this code locally, then make a directory named "Data" within the present working directory and change the paths. Also, you need to install the required libraries. You can do this by running `pip install mediapipe numpy opencv-python` in the terminal. Once the program is running, enter the name of the gesture and press enter. Once this is done, your camera will open and you will have to perform the gesture. The program will then save the data in the "Data" directory, in the form of npy file. You can then use this data to train a model.

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

# Initialize Mediapipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

folder = "D:\Krishna\SIGNAURA\SignAura\Gesture Recognition\Data"
counter = 0

# Create the dataset directory
gesture_name = input("Enter the gesture name: ")
path = os.path.join(folder, gesture_name)
if not os.path.exists(path):
    os.makedirs(path)

cap = cv2.VideoCapture(1)

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks on the image
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Collect landmark data
            landmark_list = []
            for id, lm in enumerate(hand_landmarks.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                landmark_list.append([cx, cy])

            # Save the landmark data as a numpy array for training
            np.save(os.path.join(path, f"landmark_{counter}.npy"), np.array(landmark_list))
            counter += 1

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


<>:11: SyntaxWarning: invalid escape sequence '\K'
<>:11: SyntaxWarning: invalid escape sequence '\K'
C:\Users\Krishna Moghe\AppData\Local\Temp\ipykernel_32516\266379155.py:11: SyntaxWarning: invalid escape sequence '\K'
  folder = "D:\Krishna\SIGNAURA\SignAura\Gesture Recognition\Data"
C:\Users\Krishna Moghe\AppData\Local\Temp\ipykernel_32516\266379155.py:11: SyntaxWarning: invalid escape sequence '\K'
  folder = "D:\Krishna\SIGNAURA\SignAura\Gesture Recognition\Data"


KeyboardInterrupt: 

### **Model Training**
The code is designed to train a gesture recognition model using landmark data collected from hand gestures.
<br>
**Model Used**: Support Vector Classifier (SVC) with a linear kernel.<br>
**Why SVC with a Linear Kernel?**: The linear kernel is appropriate for this gesture recognition task, assuming the hand landmarks (features) are linearly separable. SVM is also known for its ability to handle high-dimensional data effectively and prevent overfitting, which is important for gesture recognition with landmark data.

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle

# Set the absolute path for the data folder
data_folder = "D:/Krishna/SIGNAURA/SignAura/Gesture Recognition/Data"  # Use absolute path to avoid relative path issues

# Check if the data folder exists
if not os.path.exists(data_folder):
    print(f"Error: The directory '{data_folder}' does not exist.")
    exit()

# List all subdirectories (gesture classes) in the data folder
classes = os.listdir(data_folder)

# Initialize data and labels lists
data = []
labels = []

# Loop through each class (gesture folder)
for idx, class_name in enumerate(classes):
    class_path = os.path.join(data_folder, class_name)
    
    # Check if the directory exists
    if not os.path.isdir(class_path):
        print(f"Error: '{class_path}' is not a valid directory.")
        continue
    
    # Check if the folder contains any landmark files
    landmark_files = os.listdir(class_path)
    if len(landmark_files) == 0:
        print(f"Warning: The folder '{class_path}' is empty.")
        continue

    # Loop through each landmark file in the class folder
    for landmark_file in landmark_files:
        file_path = os.path.join(class_path, landmark_file)
        
        # Try to load the landmark data
        try:
            landmark = np.load(file_path)
            data.append(landmark.flatten())
            labels.append(idx)
        except Exception as e:
            print(f"Error loading file '{file_path}': {e}")
            continue

# Convert data and labels to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Check if data is empty after processing all folders
if len(data) == 0:
    print("Error: No data available for training. Please check your folders and files.")
    exit()

# Split data into training and testing sets (80% train, 20% test)
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Train the model using a Support Vector Classifier (SVC) with a linear kernel
model = SVC(kernel='linear')
model.fit(x_train, y_train)

# Evaluate the model on the test set
accuracy = model.score(x_test, y_test)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Save the trained model to a file using pickle
model_filename = "gesture_model.pkl"
with open(model_filename, "wb") as f:
    pickle.dump(model, f)
    print(f"Model saved to '{model_filename}'.")



### **Gesture Recognition**
- This program captures video from a webcam and uses Mediapipe to detect hand landmarks.
- The hand landmark data is passed to a pre-trained Support Vector Classifier (SVC) model to predict the corresponding gesture (e.g., a number from 0 to 9).
- The predicted gesture is displayed in real-time on the video feed, providing an interactive and dynamic gesture recognition system.

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pickle

# Load model
with open("D:\Krishna\SIGNAURA\SignAura\Gesture Recognition\gesture_model.pkl", "rb") as f:
    model = pickle.load(f)

# Initialize Mediapipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils

# Labels for digits 0-9
# labels = ["૦","૧","૨","૩","૪","૫","૬","૭","૮","૯"]
labels = [str(i) for i in range(10)]
cap = cv2.VideoCapture(1)

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks on the image
            mp_draw.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Collect landmark data
            landmark_list = []
            for id, lm in enumerate(hand_landmarks.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                landmark_list.append([cx, cy])

            # Prediction
            landmark_flatten = np.array(landmark_list).flatten().reshape(1, -1)
            prediction = model.predict(landmark_flatten)
            predicted_label = labels[prediction[0]]

            cv2.putText(img, predicted_label, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)

    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

